In [ ]:
# using Pkg
# Pkg.add(url = "https://github.com/JuliaReliab/JuliaMBD.git", rev="develop")
using Pkg
Pkg.update()

In [ ]:
# using Pkg
# Pkg.add("DifferentialEquations")
# using Pkg
# Pkg.add("Plots")
using DifferentialEquations
using Plots

In [ ]:
using JuliaMBD
import JuliaMBD: expr, expr_initial, get_inports, get_outports

In [ ]:
module MyModule

using JuliaMBD
import JuliaMBD: expr, expr_initial, get_inports, get_outports

@model MSD begin
    @parameter begin
        M::Float64
        D::Float64
        k::Float64
        g = 9.8
    end
    @block begin
        in1 = InBlock(:in1)
        out1 = OutBlock(:out1)
        constant1 = Constant(:(M*g))
        gain1 = Gain(:D)
        gain2 = Gain(:k)
        gain3 = Gain(:(1/M))
        int1 = Integrator(:s1)
        int2 = Integrator(:s2, initialcondition=:(M*g/k))
        add = Add(inports=[InPort(), InPort(), InPort(), InPort()], signs=[:+, :+, :-, :-])
    end
    
    in1 => add.inports[1]
    constant1 => add.inports[2]
    gain1 => add.inports[3]
    gain2 => add.inports[4]
    add => gain3 => int1 => [int2, gain1]
    int2 => [gain2, out1]
end

 @model TestBlock begin
    @parameter begin
        M::Float64
        D::Float64
        k::Float64
        f::Float64
        p_cycle::Float64
        p_width::Float64
    end
    @block begin
        msd = MSD(out1=OutPort(:x))
        pulse = PulseGenerator(amplitude=:f, period=:p_cycle, pulsewidth=:p_width, phasedelay=10.0, outport=OutPort(:F))
    end

    @scope begin
        pulse
        msd.out1
    end

    pulse => msd.in1
end

end

In [ ]:
m = Main.MyModule.TestBlock(M=10.0, D=10.0, k=18.0, f=1.0, p_cycle=20.0, p_width=50.0)

In [ ]:
@time results = simulate(m, (0, 100.0), n=100);

In [ ]:
results.graph